In [42]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn as nn

from IPython.display import Image
from torchvision.utils import save_image

mnist = MNIST(root='./../data', 
              train=True, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [43]:
image_size = 784
hidden_size = 256
latent_size = 64

batch_size = 100
data_loader = DataLoader(mnist, batch_size, shuffle=True)

In [44]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [45]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [46]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

def train_discriminator(images):
    real_labels = torch.ones(batch_size, 1)
    fake_labels = torch.zeros(batch_size, 1)
        
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    z = torch.randn(batch_size, latent_size)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()
    
    return d_loss, real_score, fake_score

In [47]:
def train_generator():
    z = torch.randn(batch_size, latent_size)
    fake_images = G(z)
    labels = torch.ones(batch_size, 1)
    g_loss = criterion(D(fake_images), labels)

    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images

In [48]:
import os

sample_dir = './../data/samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [49]:
sample_vectors = torch.randn(batch_size, latent_size)

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'simple_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=10)

In [ ]:
%%time

num_epochs = 300
total_step = len(data_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # Load a batch & transform to vectors
        images = images.reshape(batch_size, -1)
        
        # Train the discriminator and generator
        d_loss, real_score, fake_score = train_discriminator(images)
        g_loss, fake_images = train_generator()
        
        # Inspect the losses
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # Sample and save images
    save_fake_images(epoch+1)

Epoch [0/300], Step [200/600], d_loss: 0.0371, g_loss: 4.3454, D(x): 0.99, D(G(z)): 0.03
Epoch [0/300], Step [400/600], d_loss: 0.0926, g_loss: 5.5240, D(x): 0.97, D(G(z)): 0.05
Epoch [0/300], Step [600/600], d_loss: 0.0420, g_loss: 5.5349, D(x): 0.99, D(G(z)): 0.03
Saving simple_images-0001.png
Epoch [1/300], Step [200/600], d_loss: 0.2197, g_loss: 4.7048, D(x): 0.97, D(G(z)): 0.16
Epoch [1/300], Step [400/600], d_loss: 0.0960, g_loss: 4.4969, D(x): 0.98, D(G(z)): 0.07
Epoch [1/300], Step [600/600], d_loss: 0.1672, g_loss: 4.0754, D(x): 0.96, D(G(z)): 0.10
Saving simple_images-0002.png
Epoch [2/300], Step [200/600], d_loss: 0.3610, g_loss: 3.2462, D(x): 0.85, D(G(z)): 0.11
Epoch [2/300], Step [400/600], d_loss: 0.1589, g_loss: 4.4309, D(x): 0.91, D(G(z)): 0.02
Epoch [2/300], Step [600/600], d_loss: 0.4826, g_loss: 2.6092, D(x): 0.87, D(G(z)): 0.23
Saving simple_images-0003.png
Epoch [3/300], Step [200/600], d_loss: 0.4672, g_loss: 3.4377, D(x): 0.80, D(G(z)): 0.09
Epoch [3/300], Step 

Epoch [27/300], Step [600/600], d_loss: 0.4512, g_loss: 2.7865, D(x): 0.89, D(G(z)): 0.14
Saving simple_images-0028.png
Epoch [28/300], Step [200/600], d_loss: 0.2880, g_loss: 4.2501, D(x): 0.94, D(G(z)): 0.14
Epoch [28/300], Step [400/600], d_loss: 0.3182, g_loss: 3.5520, D(x): 0.89, D(G(z)): 0.09
Epoch [28/300], Step [600/600], d_loss: 0.5104, g_loss: 2.7652, D(x): 0.87, D(G(z)): 0.16
Saving simple_images-0029.png
Epoch [29/300], Step [200/600], d_loss: 0.3885, g_loss: 4.5143, D(x): 0.86, D(G(z)): 0.09
Epoch [29/300], Step [400/600], d_loss: 0.6863, g_loss: 3.8187, D(x): 0.77, D(G(z)): 0.07
Epoch [29/300], Step [600/600], d_loss: 0.4075, g_loss: 3.4432, D(x): 0.90, D(G(z)): 0.13
Saving simple_images-0030.png
Epoch [30/300], Step [200/600], d_loss: 0.4266, g_loss: 3.3444, D(x): 0.92, D(G(z)): 0.18
Epoch [30/300], Step [400/600], d_loss: 0.5135, g_loss: 4.0763, D(x): 0.86, D(G(z)): 0.13
Epoch [30/300], Step [600/600], d_loss: 0.5298, g_loss: 3.0918, D(x): 0.89, D(G(z)): 0.20
Saving sim

Epoch [55/300], Step [200/600], d_loss: 0.5931, g_loss: 3.0766, D(x): 0.78, D(G(z)): 0.13
Epoch [55/300], Step [400/600], d_loss: 0.6607, g_loss: 2.4178, D(x): 0.78, D(G(z)): 0.19
Epoch [55/300], Step [600/600], d_loss: 0.4368, g_loss: 3.3810, D(x): 0.83, D(G(z)): 0.11
Saving simple_images-0056.png
Epoch [56/300], Step [200/600], d_loss: 0.5504, g_loss: 3.2136, D(x): 0.80, D(G(z)): 0.17
Epoch [56/300], Step [400/600], d_loss: 0.9064, g_loss: 2.3008, D(x): 0.81, D(G(z)): 0.34
Epoch [56/300], Step [600/600], d_loss: 0.5848, g_loss: 2.8601, D(x): 0.83, D(G(z)): 0.22
Saving simple_images-0057.png
Epoch [57/300], Step [200/600], d_loss: 0.5240, g_loss: 3.3192, D(x): 0.76, D(G(z)): 0.11
Epoch [57/300], Step [400/600], d_loss: 0.4894, g_loss: 3.5535, D(x): 0.78, D(G(z)): 0.08
Epoch [57/300], Step [600/600], d_loss: 0.5072, g_loss: 2.7945, D(x): 0.79, D(G(z)): 0.11
Saving simple_images-0058.png
Epoch [58/300], Step [200/600], d_loss: 0.5525, g_loss: 1.9979, D(x): 0.81, D(G(z)): 0.20
Epoch [58/

In [32]:
import cv2
import os
from IPython.display import FileLink

vid_fname = sample_dir + '/gans_training.avi'

files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'fake_images' in f]
files.sort()

out = cv2.VideoWriter(vid_fname,cv2.VideoWriter_fourcc(*'MP4V'), 8, (302,302))
[out.write(cv2.imread(fname)) for fname in files]
out.release()
FileLink(vid_fname)

/workspace/fl/data/samples/gans_training.avi

In [ ]:
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');